In [ ]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 9.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=6abe028157ba084763a2d24de09be85a7897385066dd6c23c88f041996ff4bff
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [ ]:
!pip install jsonlines

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize.treebank import TreebankWordTokenizer, TreebankWordDetokenizer
from googletrans import Translator
from tqdm import tqdm

df = pd.read_json('dev.jsonlines', lines=True)

detokenizer = TreebankWordDetokenizer()
tokenizer = TreebankWordTokenizer()
translator = Translator()

src_lang = "en"
dest_lang = "fr"



In [ ]:
def sublist_index(sub, lst):
    sub_len = len(sub)
    for i in range(len(lst)):
        if lst[i:i + sub_len] == sub:
            return (i, i + sub_len - 1)
    return None

In [ ]:
def get_mapping(src_start, src_end, index_mappings):
  for mapping in index_mappings:
    if mapping['src_start'] == src_start and mapping['src_end'] == src_end:
      return mapping['dest_start'], mapping['dest_end']

In [ ]:
count = 0

trows = []

for index, row in tqdm(df.iterrows()):
  trans_row = {}
  index_mappings = []
  tokenized_sentence_array = []
  trans_tok_sen_array = []
  sentences = row.sentences
  translated_sentences = []
  ent_spans = row.ent_spans
  gold_evt_links = row.gold_evt_links
  evt_triggers = row.evt_triggers

  for sentence in sentences:
    tokenized_sentence_array = tokenized_sentence_array + sentence
    actual_sentence = detokenizer.detokenize(sentence)
    translated_sentence = translator.translate(actual_sentence, dest_lang, src_lang).text
    trans_token_sen = tokenizer.tokenize(translated_sentence)
    translated_sentences.append(trans_token_sen)
    trans_tok_sen_array = trans_tok_sen_array + trans_token_sen

  # print(translated_sentences, rel_triggers, ent_spans)
  spans_found = True
  trans_ent_spans = []
  for span in ent_spans:
    span_start = span[0]
    span_end = span[1] + 1
    span_type = span[2][0]
    tokenized_arg = tokenized_sentence_array[span_start: span_end]
    trans_tok_arg = tokenizer.tokenize(translator.translate(detokenizer.detokenize(tokenized_arg), dest_lang, src_lang).text)
    if sublist_index(trans_tok_arg, trans_tok_sen_array) != None:
      trans_span_start, trans_span_end = sublist_index(trans_tok_arg, trans_tok_sen_array)
      trans_ent_spans.append([trans_span_start, trans_span_end, [span_type]])
      translated_index_mapping = {'src_start': span_start, 'src_end': span_end-1, 'dest_start': trans_span_start, 'dest_end': trans_span_end}
      for mapping in index_mappings:
        if mapping['src_start'] == span_start and mapping['src_end'] == span_end-1 and mapping['dest_start'] == trans_span_start and mapping['dest_end'] == trans_span_end:
          continue # Mapping already exists, so do not append
      else:
        index_mappings.append(translated_index_mapping)
    else:
      spans_found = False

  triggers_found = True
  trans_evt_triggers = []
  for trigger in evt_triggers:
    trigger_start = trigger[0]
    trigger_end = trigger[1] + 1
    trigger_type = trigger[2][0]
    tok_trigger = tokenized_sentence_array[trigger_start: trigger_end]
    trans_tok_trigger = tokenizer.tokenize(translator.translate(detokenizer.detokenize(tok_trigger), dest_lang, src_lang).text)
    if sublist_index(trans_tok_trigger, trans_tok_sen_array) != None:
      trans_trigger_start, trans_trigger_end = sublist_index(trans_tok_trigger, trans_tok_sen_array)
      trans_evt_triggers.append([trans_trigger_start, trans_trigger_end, [trigger_type]])
      translated_index_mapping = {'src_start': trigger_start, 'src_end': trigger_end-1, 'dest_start': trans_trigger_start, 'dest_end': trans_trigger_end}
      for mapping in index_mappings:
        if mapping['src_start'] == trigger_start and mapping['src_end'] == trigger_end-1 and mapping['dest_start'] == trans_trigger_start and mapping['dest_end'] == trans_trigger_end:
          continue # Mapping already exists, so do not append
      else:
        index_mappings.append(translated_index_mapping)
    else:
      triggers_found = False

  trans_g_evt_links = []
  if(spans_found and triggers_found):
    for gevt_links in gold_evt_links:
      g_trigger_start = gevt_links[0][0]
      g_trigger_end = gevt_links[0][1]
      g_trans_trig_start, g_trans_trig_end = get_mapping(g_trigger_start, g_trigger_end, index_mappings)
      g_arg_start = gevt_links[1][0]
      g_arg_end = gevt_links[1][1]
      g_trans_arg_start, g_trans_arg_end = get_mapping(g_arg_start, g_arg_end, index_mappings)
      g_arg_type = gevt_links[2]
      trans_g_evt_links.append([[g_trans_trig_start, g_trans_trig_end], [g_trans_arg_start, g_trans_arg_end], g_arg_type])

    trans_row['rel_triggers'] = row.rel_triggers
    trans_row['gold_rel_links'] = row.gold_rel_links
    trans_row['doc_key'] = row.doc_key
    trans_row['language_id'] = dest_lang
    trans_row['source_url'] = row.source_url
    trans_row['split'] = row.split
    trans_row['sentences'] = translated_sentences
    trans_row['ent_spans'] = trans_ent_spans
    trans_row['evt_triggers'] = trans_evt_triggers
    trans_row['gold_evt_links'] = trans_g_evt_links

    trows.append(trans_row)


    count = count + 1
print()
print(count)

924it [13:46,  1.12it/s]


33


In [ ]:
import jsonlines

filename = 'dev.fr.jsonlines'

with jsonlines.open(filename, mode='w') as writer:
    writer.write_all(trows)

In [ ]:
from google.colab import files

files.download('dev.fr.jsonlines')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>